In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Imports 

In [ ]:
# These are some basic packages
import random, re, math, os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


# These are for data processing
import tensorflow_addons as tfa
from kaggle_datasets import KaggleDatasets


# These are for model training
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Input, Conv2D,MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,GlobalAveragePooling2D,  Activation, concatenate, Add
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
 

# These are for class weights
import datetime
import tqdm
import json
from collections import Counter
import gc
# These are performance metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


print("Tensorflow version " + tf.__version__)

# Load the Dataset

In [ ]:

# Check if GPUs are available
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Set GPU memory growth to True to allocate GPU memory dynamically
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        strategy = tf.distribute.MirroredStrategy()  # Use MirroredStrategy for multi-GPU training
        print("Running on ", len(gpus), " GPU(s)")
    except RuntimeError as e:
        print(e)
        strategy = tf.distribute.get_strategy()  # Default strategy if GPUs are not available
else:
    strategy = tf.distribute.get_strategy()  # Default strategy if GPUs are not available

print("REPLICAS: ", strategy.num_replicas_in_sync)


# Configuration 

In [ ]:
IMAGE_SIZE = [224, 224]
EPOCHS = 10
SEED = 100
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-224x224'

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

# Dataset

# Add more mixed precision and/or XLA (refer to Chris Deotte's notebook) [Rotation Augmentation GPU/TPU - [0.96+]](https://www.kaggle.com/code/cdeotte/rotation-augmentation-gpu-tpu-0-96/notebook) 

In [ ]:
# and can speed up the training process
MIXED_PRECISION = False
XLA_ACCELERATE = False

if MIXED_PRECISION:
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')
    

# All 104 classes 

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose'] # 100 - 103

# Visualization

# Showing the beautiful flowers . referance : Flower with TPUs - [Advanced augmentations](https://www.kaggle.com/dimitreoliveira/flower-with-tpus-advanced-augmentations)

In [ ]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object:
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target_(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red = False, titlesize = 16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize = int(titlesize) if not red else int(titlesize / 1.2), color = 'red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2] + 1)

def display_batch_of_images(databatch, predictions = None):
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
    rows = int(math.sqrt(len(images)))
    cols = len(images) // rows
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot = (rows,cols,1)
    if rows < cols:
        plt.figure(figsize = (FIGSIZE, FIGSIZE / cols*rows))
    else:
        plt.figure(figsize = (FIGSIZE / rows * cols,FIGSIZE))
        
    # Display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target_(predictions[i], label)
        dynamic_titlesize = FIGSIZE * SPACING / max(rows,cols) * 40 + 3
        subplot = display_one_flower(image, title, subplot, not correct, titlesize = dynamic_titlesize)
    
    # Layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace = 0, hspace = 0)
    else:
        plt.subplots_adjust(wspace = SPACING, hspace = SPACING)
    plt.show()
    
# Visualize model predictions (on training and validation sets)
def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis = -1)
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', should be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct


def display_one_flower_eval(image, title, subplot, red = False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize = 14, color = 'red' if red else 'black')
    return subplot + 1
def display_9_images_with_predictions(images, predictions, labels):
    subplot = 331
    plt.figure(figsize = (13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower_eval(image, title, subplot, not correct)
        if i >= 8:
            break;
    plt.tight_layout()
    plt.subplots_adjust(wspace = 0.1, hspace = 0.1)
    plt.show()

# getting train, validation and test dataset 

Convert pixels to floata (0,1)

In [ ]:
def decode_image(image_data):
    
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.cast(image, tf.float32) / 255.0
    # Reshape the images to fit the size required by TPU
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image

In [ ]:
def read_labeled_tfrecord(example):
    
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    
    return image, label

# This is for data visualization
def read_labeled_id_tfrecord(example):
    
    LABELED_ID_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, LABELED_ID_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    idnum =  example['id']
    
    return image, label, idnum

read unlabeled tfrec files (i.e. test set)

In [ ]:
def read_unlabeled_tfrecord(example):
    
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    
    return image, idnum

# Loading the image data 

In [ ]:
def load_dataset(filenames, labeled = True, ordered = False):
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False  # Disable order to increase running speed
    # Automatically interleaves reading
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # Use data in the shuffled order
    dataset = dataset.with_options(ignore_order)
    # Returns a dataset of (image, label) pairs if labeled = True (i.e. training & validation set)
    # or (image, id) pair if labeld = False (i.e. test set)
    dataset = dataset.map(read_labeled_id_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    
    return dataset

# Data Augmentation 

In [ ]:
def data_augment(image, label):
    # Set seed for data augmentation
    seed = 100
    # Define wrapper functions for each transformation
    def resize_and_crop(image):
        resized_image = tf.image.resize(image, [331, 331])
        return tf.image.random_crop(resized_image, [224, 224, 3], seed=seed)

    def adjust_brightness(image):
        return tf.image.random_brightness(image, 0.6, seed=seed)

    def adjust_saturation(image):
        return tf.image.random_saturation(image, 3, 5, seed=seed)

    def adjust_contrast(image):
        return tf.image.random_contrast(image, 0.3, 0.5, seed=seed)

    def blur_image(image):
        return tfa.image.mean_filter2d(image, filter_shape=10)

    def flip_image(image):
        flipped_image = tf.image.random_flip_left_right(image, seed=seed)
        return tf.image.random_flip_up_down(flipped_image, seed=seed)

    # Apply transformations
    image = resize_and_crop(image)
    image = adjust_brightness(image)
    image = adjust_saturation(image)
    image = adjust_contrast(image)
    image = blur_image(image)
    image = flip_image(image)

    return image, label

# get the training set 

In [ ]:
def get_training_dataset():
   
    train = load_dataset(TRAINING_FILENAMES, labeled = True)
    train = train.map(lambda image, label, idnum: [image, label])
    train = train.repeat()
    train = train.shuffle(2048)
    train = train.batch(BATCH_SIZE)
    train = train.prefetch(AUTO)
    
    return train

# This function is for data visualization
def get_training_dataset_preview(ordered = True):
    
    train = load_dataset(TRAINING_FILENAMES, labeled = True, ordered = ordered)
    train = train.batch(BATCH_SIZE)
    train = train.cache()
    train = train.prefetch(AUTO)
    
    return train


# Get the Validation set 

In [ ]:
def get_validation_dataset(ordered = False):

    validation = load_dataset(VALIDATION_FILENAMES, labeled = True, ordered = ordered)
    validation = validation.map(lambda image, label, idnum: [image, label])
    validation = validation.batch(BATCH_SIZE)
    validation = validation.cache()
    # Prefetch next batch while training (autotune prefetch buffer size)
    validation = validation.prefetch(AUTO)
    
    return validation

# Get the testing set 

In [ ]:
def get_test_dataset(ordered = False):
    
    test = load_dataset(TEST_FILENAMES, labeled = False, ordered = ordered)
    test = test.batch(BATCH_SIZE)
    test = test.prefetch(AUTO)
    
    return test

# Total Number of images 


In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)


NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)  # Number of images in training set
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)  # Number of images in validation set
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)  # Number of images in test set
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE  # Steps of each epoch
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

Let see some flowers in traning dataset before augmentation.

In [ ]:
train_dataset_aug = get_training_dataset()
display_batch_of_images(next(iter(train_dataset_aug.unbatch().batch(10))))
display_batch_of_images(next(iter(train_dataset_aug.unbatch().batch(10))))
display_batch_of_images(next(iter(train_dataset_aug.unbatch().batch(10))))

# Let see flowers in val dataset begore augmentation 

In [ ]:
validation_dataset_aug = get_validation_dataset()
display_batch_of_images(next(iter(validation_dataset_aug.unbatch().batch(10))))
display_batch_of_images(next(iter(validation_dataset_aug.unbatch().batch(10))))
display_batch_of_images(next(iter(validation_dataset_aug.unbatch().batch(10))))

# Similarly in test data 

In [ ]:
    test_dataset_aug = get_test_dataset()
display_batch_of_images(next(iter(test_dataset_aug.unbatch().batch(10))))
display_batch_of_images(next(iter(test_dataset_aug.unbatch().batch(10))))
display_batch_of_images(next(iter(test_dataset_aug.unbatch().batch(10))))

# Let see example of augmentation 

In [ ]:
row = 2
col = 4
all_elements = get_training_dataset().unbatch()
one_element = tf.data.Dataset.from_tensors(next(iter(all_elements)))
# Map the images to the data augmentation function for image processing
augmented_element = one_element.repeat().map(data_augment).batch(row * col)

for (img, label) in augmented_element:
    plt.figure(figsize = (15, int(15 * row / col)))
    for j in range(row * col):
        plt.subplot(row, col, j + 1)
        plt.axis('off')
        plt.imshow(img[j, ])
    plt.show()
    break

# Model Building 

Customize the learning rate 

In [ ]:
def lrfn(epoch):
    
    LR_START = 0.00001
    LR_MAX = 0.00005 * strategy.num_replicas_in_sync
    LR_MIN = 0.00001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_EXP_DECAY = .8
    
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY ** (epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

Lets visualize it .

In [ ]:
epoch_range = range(min(EPOCHS, 25))
learning_rates = [lrfn(epoch) for epoch in epoch_range]

# Plot the learning rates
plt.figure(figsize=(6, 4))
plt.plot(epoch_range, learning_rates)
plt.title("Learning Rate Schedule")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.grid(True)
plt.show()

# Print the initial, maximum, and final learning rates
print(f"Learning rate schedule: {learning_rates[0]:.3g} to {max(learning_rates):.3g} to {learning_rates[-1]:.3g}")


# Most important part : Build the model 

In [ ]:
with strategy.scope():
    # Create DenseNet(201) model
    rnet = DenseNet201(
        input_shape = (224, 224, 3),
        weights = 'imagenet',  # Use the preset parameters of ImageNet
        include_top = False  # Drop the fully connected network on the top
    )
    
    rnet.trainable = True
    model = tf.keras.Sequential([
        rnet,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES), activation = 'softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(), 
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['sparse_categorical_accuracy']
    )

    model.summary()
    # Save the model
    model.save('DenseNet.h5')

Since classes may not be uniformly distributed  add weights

In [ ]:
gc.enable()

def get_training_dataset_raw():
    dataset = load_dataset(TRAINING_FILENAMES, labeled = True, ordered = False)
    return dataset

raw_training_dataset = get_training_dataset_raw()

label_counter = Counter()
for images, labels, id in raw_training_dataset:
    label_counter.update([labels.numpy()])
    
del raw_training_dataset    

TARGET_NUM_PER_CLASS = 122

def get_weight_for_class(class_id):
    counting = label_counter[class_id]
    weight = TARGET_NUM_PER_CLASS / counting
    return weight

weight_per_class = {class_id: get_weight_for_class(class_id) for class_id in range(104)}

In [ ]:
history = model.fit(
    get_training_dataset(),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = 30,
    callbacks = [lr_callback],
    validation_data = get_validation_dataset(),
    class_weight = weight_per_class
)

In [ ]:
# Performance 
performance_model = model.evaluate(get_validation_dataset())

# Plotting the result 

In [ ]:
# Plotting
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Checking the model performance on the validation dataset 

In [ ]:
cmdataset = get_validation_dataset(ordered = True)
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch() 
cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy()  # Get everything as one batch
cm_probabilities = model.predict(images_ds)  # The probability that each image is of each class
cm_predictions = np.argmax(cm_probabilities, axis = -1)  # The class of the largest probability is what we need
print("Correct labels: ", cm_correct_labels.shape, cm_correct_labels)
print("Predicted labels: ", cm_predictions.shape, cm_predictions)

In [ ]:
cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

# Prediction 

In [ ]:
test_ds = get_test_dataset(ordered = True)

test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)  # Compute the probability that each image is of each class
predictions = np.argmax(probabilities, axis = -1)  # Use the one with largest probability as the predicted class
print(predictions)

# Generate submission file, remember to name it by "submission.csv"
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')
test = pd.DataFrame({"id": test_ids, "label": predictions})
print(test.head)
test.to_csv("submission.csv",index = False)

# Experiment with different model 


Model with 6 layers 

In [ ]:
with strategy.scope():
    model6 = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = (224, 224, 3)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

    model6.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model6.summary()
    # Save the model
    model6.save('model6.h5')


In [ ]:
history6 = model6.fit(
    get_training_dataset(),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    callbacks = [lr_callback],
    validation_data = get_validation_dataset(),
    class_weight = weight_per_class
)

In [ ]:
with strategy.scope():
    model2 = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = (224, 224, 3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

    model2.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])
    model2.summary()
    # Save the model
    model2.save('model2.h5')


In [ ]:
history2 = model2.fit(
    get_training_dataset(),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = 20,
    callbacks = [lr_callback],
    validation_data = get_validation_dataset(),
    class_weight = weight_per_class
)

# Performance 

In [ ]:
performance_model2 =  model2.evaluate(get_validation_dataset())

# Pretrained VGG16 model and add some layers 

In [ ]:
from tensorflow.keras.applications import VGG16

with strategy.scope():
  ## Model Load 
    base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model_vgg16.trainable = True

    x = Flatten()(base_model_vgg16.output)  
    x = Dense(1024, activation='relu')(x))
    x = Dense(512, activation='relu')(x)
    output = Dense(len(CLASSES), activation='softmax')(x)

    model_vgg16 = Model(base_model_vgg16.input, output)

    model_vgg16.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    model_vgg16.summary()

In [ ]:
history_mvgg16 = model_vgg16.fit(
    get_training_dataset(),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    callbacks = [lr_callback],
    validation_data = get_validation_dataset(),
    class_weight = weight_per_class
)

# Performance 

In [ ]:
performance_vgg16 = model_vgg16.evaluate(get_validation_dataset())

# Visualization 

In [ ]:
# Plotting
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history_mvgg16.history['sparse_categorical_accuracy'])
plt.plot(history_mvgg16.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history_mvgg16.history['loss'])
plt.plot(history_mvgg16.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Pretrained Mobilenetv2 with some layers 

In [ ]:
from tensorflow.keras.applications import MobileNetV2
with strategy.scope():
# Model Loading 
    base_model_mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


    base_model_mobilenetv2.trainable = True

    x = Flatten()(base_model_mobilenetv2.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(len(CLASSES), activation='softmax')(x)

    model_mobilenetv2 = Model(base_model_mobilenetv2.input, output)

    model_mobilenetv2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    model_mobilenetv2.summary()

In [ ]:
history_mmbv = model_mobilenetv2.fit(
    get_training_dataset(),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = 20,
    callbacks = [lr_callback],
    validation_data = get_validation_dataset(),
    class_weight = weight_per_class
)

# Performance 

In [ ]:
performance_mmbv = model_mobilenetv2.evaluate(get_validation_dataset())

# Plotting 

In [ ]:
# Plotting
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history_mmbv.history['sparse_categorical_accuracy'])
plt.plot(history_mmbv.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history_mmbv.history['loss'])
plt.plot(history_mmbv.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Bar Plotting 

In [ ]:
accuracy_1 = performance_1[1]
accuracy_2 = performance_model2[1]
accuracy_3 = performance_vgg16[1]
accuracy_4 = performance_mmbv[1]

# Creating a bar chart
labels = ['Model 1', 'Model 2', 'Model 3', 'Model 4']
accuracies = [accuracy_1, accuracy_2, accuracy_3, accuracy_4]

plt.bar(labels, accuracies, color=['blue', 'green', 'red', 'purple'])
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.ylim([0, 1])  # Assuming accuracy metric, set y-axis range from 0 to 1
plt.show()